# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_df = pd.read_csv("../WeatherPy/output_data/weather_data.csv")
vacation_df.head()

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,hermanus,100,ZA,1580258943,97,-34.42,19.23,64.99,1.01
1,1,kapaa,1,US,1580258844,61,22.08,-159.32,80.60,6.93
2,2,vaini,75,TO,1580258943,79,-21.20,-175.20,84.20,5.82
3,3,umm kaddadah,0,SD,1580258943,19,13.60,26.69,57.13,10.67
4,4,rikitea,100,PF,1580258793,80,-23.12,-134.97,79.59,10.31


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = vacation_df[['Lat', 'Lng']]
weight = vacation_df['Humidity'].astype(float)

In [38]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
ideal_temp = vacation_df.loc[(vacation_df['Max Temp'] <80) & (vacation_df['Max Temp'] >70)]
ideal_wind = ideal_temp.loc[ideal_temp['Wind Speed'] >10]
ideal_cloud = ideal_wind.loc[ideal_wind['Cloudiness']>20] 
ideal_cloud

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
4,4,rikitea,100,PF,1580258793,80,-23.12,-134.97,79.59,10.31
6,6,neuquen,50,AR,1580258943,20,-38.95,-68.06,77.00,13.87
10,11,hobart,75,AU,1580258736,43,-42.88,147.33,73.40,16.11
32,35,jamestown,57,SH,1580258948,79,-15.94,-5.72,74.73,11.95
34,38,new norfolk,75,AU,1580258852,43,-42.78,147.06,73.40,16.11


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
hotel_df = pd.DataFrame(ideal_cloud, columns=['City', 'Cloudiness', 'Country', 'Date',
                                             'Humidity', 'Lat', 'Lng', 'Max Temp',
                                             'Wind Speed'])
hotel_df['Hotel Name']=''
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
4,rikitea,100,PF,1580258793,80,-23.12,-134.97,79.59,10.31,
6,neuquen,50,AR,1580258943,20,-38.95,-68.06,77.00,13.87,
10,hobart,75,AU,1580258736,43,-42.88,147.33,73.40,16.11,
32,jamestown,57,SH,1580258948,79,-15.94,-5.72,74.73,11.95,
34,new norfolk,75,AU,1580258852,43,-42.78,147.06,73.40,16.11,


In [65]:
params = {"radius": 5000,
          "types": "restaurant",
          "key": g_key
         }

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lon = row['Lng']
    
    params['location'] = f"{lat},{lon}"
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    names = requests.get(url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = names["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing result... skipping.")

Missing result... skipping.


In [72]:
hotel_df.dropna()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
4,rikitea,100,PF,1580258793,80,-23.12,-134.97,79.59,10.31,
6,neuquen,50,AR,1580258943,20,-38.95,-68.06,77.00,13.87,El Olivo Hotel Zico Gourmet Restaurant
10,hobart,75,AU,1580258736,43,-42.88,147.33,73.40,16.11,Hotel Grand Chancellor Hobart
32,jamestown,57,SH,1580258948,79,-15.94,-5.72,74.73,11.95,Richards Travel Lodge
34,new norfolk,75,AU,1580258852,43,-42.78,147.06,73.40,16.11,Subway® Restaurant


In [67]:
hotel_df.to_csv('output_data/Hotels.csv', index=False)

In [69]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_cloud.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [75]:
# Add marker layer ontop of heat map
#creating marker locations
fig = gmaps.figure()
markers = gmaps.marker_layer(locations)
#     info_box_content={hotels, countries, cities})
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))